# read point cloud file

.xyz

There are several variations of this ASCII file format, all based on Cartesian coordinates (XYZ coordinates). XYZ formats are designed for importing and exporting geometries and are widely accepted by different point cloud processing software.

In [ ]:
import numpy as np

point_cloud = np.loadtxt('sample.xyz', skiprows=1)

.ply

PLY is a computer file format known as the Polygon File Format or the Stanford Triangle Format. It was principally designed to store three-dimensional data from 3D scanners. 

In [ ]:
from open3d import *

point_cloud = io.read_point_cloud("sample.ply")

point_cloud = np.asarray(point_cloud.points)
colors = np.asarray(point_cloud.colors)

.las

LAS is a binary format used specifically for storing LiDAR data. It’s an industry-standard format, so it’s widely used and compatible with most programs. 

In [ ]:
import laspy as lp

point_cloud = lp.file.File("sample.las", mode="r")

# separate coordinates from colours, and put them in NumPy arrays
point_cloud = np.vstack((point_cloud.x, point_cloud.y, point_cloud.z)).transpose()
pc_colors = np.vstack((point_cloud.red, point_cloud.green, point_cloud.blue)).transpose()


import pclpy

point_cloud = pclpy.read("sample.las", "PointXYZRGBA")  # PointXYZ PointXYZI PointXYZINormal PointNormal PointXYZRGBNormal PointXYZRGBA
# output of pclpy.read
# ... pc = getattr(pcl.PointCloud, point_type).from_array(*data)

to torch tensor

In [ ]:
import torch

point_cloud = torch.tensor(point_cloud)

**save point clouds**

In [ ]:
import numpy as np
import pandas as pd

from pyntcloud import PyntCloud

cloud = PyntCloud(pd.DataFrame(
    # same arguments that you are passing to visualize_pcl
    data=np.hstack((points, colors)),
    columns=["x", "y", "z", "red", "green", "blue"]))

cloud.to_file("output.ply")

In [ ]:
# save and visualize via Open3D
import open3d as o3d

pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("./filename.ply", pc)

# visualize point cloud
o3d.visualization.draw_geometries([pc])

# read mesh file

.obj

In [ ]:
# method 1
from pymesh import load_mesh

mesh = load_mesh("mesh.obj")
# PyMesh supports parsing the following formats: .obj, .ply, .off, .stl, .mesh, .node, .poly and .msh.
vertices = mesh.vertices
faces = mesh.faces

# method 2
from pytorch3d.io import load_objs_as_meshes
mesh = load_objs_as_meshes("mesh.obj", device=torch.device("cpu"))

In [ ]:
# from raw data
# vertices, faces and voxels are of type numpy.ndarray. One vertex/face/voxel per row
from pymesh import from_mesh

# for surface mesh
mesh = form_mesh(vertices, faces)

# for volume mesh
mesh = pymesh.form_mesh(vertices, faces, voxels)

In [2]:
import meshio

mesh = meshio.read('bunny.obj')

vertices = mesh.points
faces = mesh.cells_dict['triangle']
print(type(vertices),type(faces))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


source: https://pymesh.readthedocs.io/en/latest/basic.html#loading-mesh

.stl

In [ ]:
from stl import mesh

# Using an existing stl file:
stl = mesh.Mesh.from_file('some_file.stl')
# output numpy.array
stl_tensor = torch.tensor(stl)
# ??

In [ ]:
import meshio

mesh = meshio.read('some_file.stl')

**save mesh**

In [ ]:
import numpy as np
import struct
# package struct for binary string processing

def write_binary_stl(path, points):
    n = len(points) // 3

    points = np.array(points, dtype='float32').reshape((-1, 3, 3))
    normals = np.cross(points[:,1] - points[:,0], points[:,2] - points[:,0])
    normals /= np.linalg.norm(normals, axis=1).reshape((-1, 1))

    dtype = np.dtype([
        ('normal', ('<f', 3)),
        ('points', ('<f', (3, 3))),
        ('attr', '<H'),
    ])

    a = np.zeros(n, dtype=dtype)
    a['points'] = points
    a['normal'] = normals

    with open(path, 'wb') as fp:
        fp.write(b'\x00' * 80)
        fp.write(struct.pack('<I', n))
        fp.write(a.tobytes())

In [ ]:
# save via package meshio
import meshio

# example code
points = [
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0],
    [2.0, 0.0],
    [2.0, 1.0],
]
cells = [
    ("triangle", [[0, 1, 2], [1, 3, 2]]),
    ("quad", [[1, 4, 5, 3]]),
]

mesh = meshio.Mesh(
    points,
    cells,
    # Optionally provide extra data on points, cells, etc.
    point_data={"T": [0.3, -1.2, 0.5, 0.7, 0.0, -3.0]},
    # Each item in cell data must match the cells array
    cell_data={"a": [[0.1, 0.2], [0.4]]},
)
mesh.write(
    "filename.vtk",  # str, os.PathLike, or buffer/open file
    # file_format="vtk",  # optional if first argument is a path; inferred from extension
)

# Alternative with the same options
meshio.write_points_cells("filename.vtk", points, cells)

In [ ]:
# save via pymesh package
from pymesh import from_mesh

mesh = form_mesh(vertices, faces)

import pymesh
pymesh.save_mesh("filename.obj", mesh, 
                # ascii=True,
                # use_float=True,
                # attribute_name_1, attribute_name_2, ...
                )

**visualize mesh**

In [ ]:
import open3d as o3d
# mesh file types support by open3d: .ply .stl .obj .off .gltf

mesh = o3d.io.read_triangle_mesh("../../filename.ply")

mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh])

# o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)

# read SDF

usually save as a function in a Python script, and import by *import*

# visualize sdf

In [ ]:
# visualize sdf

# The code simply uses the Marching Cubes algorithm to generate a mesh from the Signed Distance Function.
# This would normally be abysmally slow in Python. However, numpy is used to evaluate the SDF on entire batches 
# of points simultaneously. Furthermore, multiple threads are used to process batches in parallel. The result 
# is surprisingly fast (for marching cubes). Meshes of adequate detail can still be quite large in terms of 
# number of triangles.
from functools import partial
from multiprocessing.pool import ThreadPool
from skimage import measure

import multiprocessing
import itertools
import numpy as np
import time

from . import progress, stl

WORKERS = multiprocessing.cpu_count()
SAMPLES = 2 ** 22
BATCH_SIZE = 32

def _marching_cubes(volume, level=0):
    verts, faces, _, _ = measure.marching_cubes(volume, level)
    return verts[faces].reshape((-1, 3))

def _cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

def _skip(sdf, job):
    X, Y, Z = job
    x0, x1 = X[0], X[-1]
    y0, y1 = Y[0], Y[-1]
    z0, z1 = Z[0], Z[-1]
    x = (x0 + x1) / 2
    y = (y0 + y1) / 2
    z = (z0 + z1) / 2
    r = abs(sdf(np.array([(x, y, z)])).reshape(-1)[0])
    d = np.linalg.norm(np.array((x-x0, y-y0, z-z0)))
    if r <= d:
        return False
    corners = np.array(list(itertools.product((x0, x1), (y0, y1), (z0, z1))))
    values = sdf(corners).reshape(-1)
    same = np.all(values > 0) if values[0] > 0 else np.all(values < 0)
    return same

def _worker(sdf, job, sparse):
    X, Y, Z = job
    if sparse and _skip(sdf, job):
        return None
        # return _debug_triangles(X, Y, Z)
    P = _cartesian_product(X, Y, Z)
    volume = sdf(P).reshape((len(X), len(Y), len(Z)))
    try:
        points = _marching_cubes(volume)
    except Exception:
        return []
        # return _debug_triangles(X, Y, Z)
    scale = np.array([X[1] - X[0], Y[1] - Y[0], Z[1] - Z[0]])
    offset = np.array([X[0], Y[0], Z[0]])
    return points * scale + offset

def _estimate_bounds(sdf):
    # TODO: raise exception if bound estimation fails
    s = 16
    x0 = y0 = z0 = -1e9
    x1 = y1 = z1 = 1e9
    prev = None
    for i in range(32):
        X = np.linspace(x0, x1, s)
        Y = np.linspace(y0, y1, s)
        Z = np.linspace(z0, z1, s)
        d = np.array([X[1] - X[0], Y[1] - Y[0], Z[1] - Z[0]])
        threshold = np.linalg.norm(d) / 2
        if threshold == prev:
            break
        prev = threshold
        P = _cartesian_product(X, Y, Z)
        volume = sdf(P).reshape((len(X), len(Y), len(Z)))
        where = np.argwhere(np.abs(volume) <= threshold)
        x1, y1, z1 = (x0, y0, z0) + where.max(axis=0) * d + d / 2
        x0, y0, z0 = (x0, y0, z0) + where.min(axis=0) * d - d / 2
    return ((x0, y0, z0), (x1, y1, z1))

def generate(
        sdf,
        step=None, bounds=None, samples=SAMPLES,
        workers=WORKERS, batch_size=BATCH_SIZE,
        verbose=True, sparse=True):

    start = time.time()

    if bounds is None:
        bounds = _estimate_bounds(sdf)
    (x0, y0, z0), (x1, y1, z1) = bounds

    if step is None and samples is not None:
        volume = (x1 - x0) * (y1 - y0) * (z1 - z0)
        step = (volume / samples) ** (1 / 3)

    try:
        dx, dy, dz = step
    except TypeError:
        dx = dy = dz = step

    if verbose:
        print('min %g, %g, %g' % (x0, y0, z0))
        print('max %g, %g, %g' % (x1, y1, z1))
        print('step %g, %g, %g' % (dx, dy, dz))

    X = np.arange(x0, x1, dx)
    Y = np.arange(y0, y1, dy)
    Z = np.arange(z0, z1, dz)

    s = batch_size
    Xs = [X[i:i+s+1] for i in range(0, len(X), s)]
    Ys = [Y[i:i+s+1] for i in range(0, len(Y), s)]
    Zs = [Z[i:i+s+1] for i in range(0, len(Z), s)]

    batches = list(itertools.product(Xs, Ys, Zs))
    num_batches = len(batches)
    num_samples = sum(len(xs) * len(ys) * len(zs)
        for xs, ys, zs in batches)

    if verbose:
        print('%d samples in %d batches with %d workers' %
            (num_samples, num_batches, workers))

    points = []
    skipped = empty = nonempty = 0
    bar = progress.Bar(num_batches, enabled=verbose)
    pool = ThreadPool(workers)
    f = partial(_worker, sdf, sparse=sparse)
    for result in pool.imap(f, batches):
        bar.increment(1)
        if result is None:
            skipped += 1
        elif len(result) == 0:
            empty += 1
        else:
            nonempty += 1
            points.extend(result)
    bar.done()

    if verbose:
        print('%d skipped, %d empty, %d nonempty' % (skipped, empty, nonempty))
        triangles = len(points) // 3
        seconds = time.time() - start
        print('%d triangles in %g seconds' % (triangles, seconds))

    return points

def save(path, *args, **kwargs):
    points = generate(*args, **kwargs)
    if path.lower().endswith('.stl'):
        stl.write_binary_stl(path, points)
    else:
        mesh = _mesh(points)
        mesh.write(path)

def _mesh(points):
    import meshio
    points, cells = np.unique(points, axis=0, return_inverse=True)
    cells = [('triangle', cells.reshape((-1, 3)))]
    return meshio.Mesh(points, cells)

def _debug_triangles(X, Y, Z):
    x0, x1 = X[0], X[-1]
    y0, y1 = Y[0], Y[-1]
    z0, z1 = Z[0], Z[-1]

    p = 0.25
    x0, x1 = x0 + (x1 - x0) * p, x1 - (x1 - x0) * p
    y0, y1 = y0 + (y1 - y0) * p, y1 - (y1 - y0) * p
    z0, z1 = z0 + (z1 - z0) * p, z1 - (z1 - z0) * p

    v = [
        (x0, y0, z0),
        (x0, y0, z1),
        (x0, y1, z0),
        (x0, y1, z1),
        (x1, y0, z0),
        (x1, y0, z1),
        (x1, y1, z0),
        (x1, y1, z1),
    ]

    return [
        v[3], v[5], v[7],
        v[5], v[3], v[1],
        v[0], v[6], v[4],
        v[6], v[0], v[2],
        v[0], v[5], v[1],
        v[5], v[0], v[4],
        v[5], v[6], v[7],
        v[6], v[5], v[4],
        v[6], v[3], v[7],
        v[3], v[6], v[2],
        v[0], v[3], v[2],
        v[3], v[0], v[1],
    ]

def sample_slice(
        sdf, w=1024, h=1024,
        x=None, y=None, z=None, bounds=None):

    if bounds is None:
        bounds = _estimate_bounds(sdf)
    (x0, y0, z0), (x1, y1, z1) = bounds

    if x is not None:
        X = np.array([x])
        Y = np.linspace(y0, y1, w)
        Z = np.linspace(z0, z1, h)
        extent = (Z[0], Z[-1], Y[0], Y[-1])
        axes = 'ZY'
    elif y is not None:
        Y = np.array([y])
        X = np.linspace(x0, x1, w)
        Z = np.linspace(z0, z1, h)
        extent = (Z[0], Z[-1], X[0], X[-1])
        axes = 'ZX'
    elif z is not None:
        Z = np.array([z])
        X = np.linspace(x0, x1, w)
        Y = np.linspace(y0, y1, h)
        extent = (Y[0], Y[-1], X[0], X[-1])
        axes = 'YX'
    else:
        raise Exception('x, y, or z position must be specified')

    P = _cartesian_product(X, Y, Z)
    return sdf(P).reshape((w, h)), extent, axes

def show_slice(*args, **kwargs):
    import matplotlib.pyplot as plt
    show_abs = kwargs.pop('abs', False)
    a, extent, axes = sample_slice(*args, **kwargs)
    if show_abs:
        a = np.abs(a)
    im = plt.imshow(a, extent=extent, origin='lower')
    plt.xlabel(axes[0])
    plt.ylabel(axes[1])
    plt.colorbar(im)
    plt.show()